In [1]:
#Part1 定义模型
import torch
torch.manual_seed(0) #固定随机数种子，确保参数（训练效果）一致
torch.backends.cudnn.deterministic = False #GPU使用默认算法
torch.backends.cudnn.benchmark = True #GPU寻找最优算法

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset

from ctypes.wintypes import INT, LONG
import os, sys, glob, shutil, json
from pickletools import long1, long4
import cv2

from PIL import Image
import numpy as np
from torch.utils.data.dataset import Dataset


# 定义模型
class SVHN_Model1(nn.Module): #输入数据为3x64x128
    def __init__(self):
        super(SVHN_Model1, self).__init__() #super()可以理解为指向父类的指针
        # CNN提取特征模块
        self.cnn = nn.Sequential( #顺序构建网络
            nn.Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2)), #in_channel, out_channel, 卷积核大小, 滑动步长
            nn.ReLU(),  #非线性激活函数，为了避免神经网络中全部是线性函数，那样就不需要神经网络了
            nn.MaxPool2d(2), #最大池化MaxPooling，大小为2x2
            nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2)),
            nn.ReLU(), 
            nn.MaxPool2d(2),
        )
        # 
        self.fc1 = nn.Linear(32*3*7, 11)
        self.fc2 = nn.Linear(32*3*7, 11) #有点问题
        self.fc3 = nn.Linear(32*3*7, 11)
        self.fc4 = nn.Linear(32*3*7, 11)
        self.fc5 = nn.Linear(32*3*7, 11)
        self.fc6 = nn.Linear(32*3*7, 11)
    
    def forward(self, img):
        feat = self.cnn(img)
        feat = feat.view(feat.shape[0], -1)
        c1 = self.fc1(feat)
        c2 = self.fc2(feat)
        c3 = self.fc3(feat)
        c4 = self.fc4(feat)
        c5 = self.fc5(feat)
        c6 = self.fc6(feat)
        return c1, c2, c3, c4, c5, c6
    

model = SVHN_Model1()

In [2]:
#Part2 读取模型
model.load_state_dict(torch.load('./model3_27.pt'))

<All keys matched successfully>

In [4]:
#Part3 模型测试
import pandas as pd
import sys
#from torchvision.transforms import ToTensor
#from torchvision.transforms import Resize

predict = []
test_glob = glob.glob('C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a/*.png') #批量读取文件名到列表
for test_path in test_glob:
    print(test_path)
    img = Image.open(test_path).convert('RGB')
    img=img.resize((128, 64))
    totensor = transforms.ToTensor()  #将PIL image或ndarray类型变换成tensor形式
    tensor0 = totensor(img)
    
    torch_norm = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    tensor0 = torch_norm(tensor0)
    tensor0 = Variable(torch.unsqueeze(tensor0, dim=0).float(), requires_grad=False)

    c0, c1, c2, c3, c4, c5 = model(tensor0)
    pre = [c0, c1, c2, c3, c4, c5]
    
    c0_pre = ((c0.tolist())[0]).index(max((c0.tolist())[0]))#debug发现是二维结构
    c1_pre = ((c1.tolist())[0]).index(max((c1.tolist())[0]))
    c2_pre = ((c2.tolist())[0]).index(max((c2.tolist())[0]))
    c3_pre = ((c3.tolist())[0]).index(max((c3.tolist())[0]))
    c4_pre = ((c4.tolist())[0]).index(max((c4.tolist())[0]))
    c5_pre = ((c5.tolist())[0]).index(max((c5.tolist())[0]))
    
    pre = [c0_pre, c1_pre, c2_pre, c3_pre, c4_pre, c5_pre]
    print(pre)
    pre.remove(10) #根据值删除元素
    #predict.append(pre)
#df = pd.DataFrame(predict)
#df.to_excel("./predict.xls")


C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\000008.png
[4, 3, 3, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\000009.png
[3, 8, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\000018.png
[4, 10, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\000019.png
[1, 3, 4, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\000020.png
[2, 2, 2, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\000021.png
[2, 7, 5, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\000024.png
[2, 10, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\000025.png
[1, 0, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\000026.png
[1, 8, 10, 10, 10, 10]
C:/Project/python/Jupyter/Prac

[1, 9, 2, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\000633.png
[5, 10, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\000634.png
[1, 9, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\000635.png
[3, 0, 8, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\000636.png
[1, 10, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\000637.png
[1, 10, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\000740.png
[2, 4, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\000741.png
[2, 0, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\000742.png
[2, 2, 5, 5, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\000743.png
[2, 8, 9, 10, 10, 10]
C:/Proj

[3, 9, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\001448.png
[2, 2, 2, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\001449.png
[5, 0, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\001458.png
[1, 9, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\001459.png
[5, 10, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\001460.png
[2, 9, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\001461.png
[1, 10, 1, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\001462.png
[5, 8, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\001463.png
[1, 2, 2, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\001464.png
[1, 3, 10, 10, 10, 10]
C:/Pro

[2, 0, 2, 8, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\002144.png
[2, 4, 4, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\002145.png
[1, 8, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\002150.png
[5, 0, 6, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\002151.png
[1, 9, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\002154.png
[2, 6, 9, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\002155.png
[1, 4, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\002156.png
[2, 7, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\002157.png
[1, 1, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\002168.png
[1, 10, 10, 10, 10, 10]
C:/Projec

[1, 0, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\002746.png
[3, 3, 4, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\002747.png
[1, 5, 9, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\002750.png
[3, 3, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\002751.png
[1, 7, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\002752.png
[1, 2, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\002753.png
[3, 2, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\002756.png
[3, 8, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\002757.png
[3, 6, 5, 5, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\002778.png
[5, 8, 9, 6, 10, 10]
C:/Project/

...

[1, 6, 5, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\036783.png
[2, 8, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\036784.png
[3, 0, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\036785.png
[1, 0, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\036790.png
[1, 0, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\036791.png
[7, 1, 4, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\036794.png
[1, 0, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\036795.png
[1, 7, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\036796.png
[2, 6, 8, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\036797.png
[1, 0, 10, 10, 10, 10]
C:/Proje

[1, 3, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\037477.png
[1, 0, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\037488.png
[1, 1, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\037489.png
[2, 10, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\037500.png
[3, 3, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\037501.png
[1, 3, 5, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\037502.png
[1, 2, 5, 6, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\037503.png
[3, 0, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\037505.png
[2, 2, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\037511.png
[4, 0, 1, 10, 10, 10]
C:/Proje

[3, 3, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\038154.png
[1, 10, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\038155.png
[3, 6, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\038156.png
[3, 7, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\038157.png
[6, 8, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\038168.png
[2, 3, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\038169.png
[2, 8, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\038178.png
[2, 2, 2, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\038180.png
[2, 9, 9, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\038181.png
[1, 10, 10, 10, 10, 10]
C:/Pr

[1, 3, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\038778.png
[5, 1, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\038779.png
[1, 2, 3, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\038780.png
[3, 8, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\038781.png
[2, 4, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\038784.png
[1, 4, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\038785.png
[1, 0, 5, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\038786.png
[3, 5, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\038787.png
[6, 6, 1, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\038790.png
[1, 0, 10, 10, 10, 10]
C:/Proje

[1, 2, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\039472.png
[1, 9, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\039473.png
[1, 7, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\039474.png
[2, 3, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\039475.png
[5, 8, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\039498.png
[3, 9, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\039499.png
[2, 8, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\039500.png
[3, 10, 10, 10, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\039504.png
[4, 5, 5, 6, 10, 10]
C:/Project/python/Jupyter/Practise/tianchi/data/mchar_test_a/mchar_test_a\039505.png
[6, 0, 10, 10, 10, 10]
C:/Pro

In [ ]:
tensor_type.shape

In [ ]:
tensor_resize.shape